In [33]:
import sqlite3
import pandas as pd

**CONNECTION ESTABLISHED**

In [34]:
con=sqlite3.connect("/content/sql-murder-mystery.db")

**1. RETRIEVE CRIME SCENE REPORT**

In [35]:
cursor=con.execute('''
  SELECT * FROM crime_scene_report
  WHERE type='murder' AND city='SQL City' AND  date= 20180115 ''')
for row in cursor:
  print(row)

(20180115, 'murder', 'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".', 'SQL City')


**2. PERSONAL DETAILS OF WITNESS**

In [36]:
cursor=con.execute('''
SELECT
    interview.transcript,
    person.name,
    person.id
    FROM interview
    JOIN person
    ON interview.person_id=person.id
    WHERE person.address_street_name='Northwestern Dr'
    ORDER BY address_number desc
    LIMIT 1''')
for row in cursor:
  print(row)

('I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".', 'Morty Schapiro', 14887)


In [37]:
cursor=con.execute('''
SELECT
    interview.transcript,
    person.name,
    person.id
    FROM interview
    JOIN person
    ON interview.person_id=person.id
    WHERE
    person.name LIKE '%Annabel%'
    AND
    person.address_street_name='Franklin Ave'
    ''')
for row in cursor:
  print(row)

('I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.', 'Annabel Miller', 16371)


In [38]:
# "Get Fit Now Gym" bag.
# membership number on the bag started with "48Z".
# Only gold members have those bags.
# car with a plate that included "H42W".
# Suspect in gym on January the 9th.

**3. TO CHECK THE INFORMATION BASED ON WITNESS INTERVIEW**

In [39]:
cursor=con.execute('''
SELECT *
    FROM get_fit_now_member
    JOIN get_fit_now_check_in
    ON get_fit_now_member.id = get_fit_now_check_in.membership_id
    WHERE
        get_fit_now_member.membership_status = 'gold'
        AND
        get_fit_now_check_in.check_in_date = 20180109
        AND
        get_fit_now_check_in.membership_id LIKE '%48Z%'
   ''' )

for row in cursor:
  print(row)

('48Z7A', 28819, 'Joe Germuska', 20160305, 'gold', '48Z7A', 20180109, 1600, 1730)
('48Z55', 67318, 'Jeremy Bowers', 20160101, 'gold', '48Z55', 20180109, 1530, 1700)


**4. CHECK THE CAR DETAILS**

In [40]:

cursor=con.execute('''
SELECT * FROM drivers_license
WHERE plate_number LIKE '%H42W%'
''')
for row in cursor:
  print(row)

(183779, 21, 65, 'blue', 'blonde', 'female', 'H42W0X', 'Toyota', 'Prius')
(423327, 30, 70, 'brown', 'brown', 'male', '0H42W2', 'Chevrolet', 'Spark LS')
(664760, 21, 71, 'black', 'black', 'male', '4H42WR', 'Nissan', 'Altima')


**5. CHECK THE PERSONAL DETAILS OF MALES FROM THE ABOVE QUERY**

In [41]:
cursor=con.execute('''
SELECT * FROM person
WHERE license_id= 423327 or license_id=664760

''')
for row in cursor:
  print(row)

(51739, 'Tushar Chandra', 664760, 312, 'Phi St', 137882671)
(67318, 'Jeremy Bowers', 423327, 530, 'Washington Pl, Apt 3A', 871539279)


**6. CHECK THE MEMBERSHIP DETAILS IN THE GYM OF TWO MALES TO FIND SUSPECT**

In [42]:
cursor=con.execute('''

SELECT * FROM get_fit_now_member
WHERE person_id=51739 or person_id= 67318
''')
for row in cursor:
  print(row)

('48Z55', 67318, 'Jeremy Bowers', 20160101, 'gold')


**7. SUSPECT INTERVIEW**

In [43]:
cursor=con.execute('''
SELECT * FROM interview
WHERE person_id = 67318
''')

for row in cursor:
  print(row)

(67318, 'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n')


In [44]:
# gender- female
# height- 5.5
# red hair, tesla model s
# attended SQL Symphony Concert 3 times in Dec 2017.

**8. CHECK THE CAR DETAILS OF A PERSON BASED ON ABOVE QUERY**

In [45]:
cursor=con.execute('''
SELECT *
FROM drivers_license
WHERE car_make = "Tesla" AND car_model = "Model S" AND
gender = "female" AND hair_color = "red";
''')

for row in cursor:
  print(row)

(202298, 68, 66, 'green', 'red', 'female', '500123', 'Tesla', 'Model S')
(291182, 65, 66, 'blue', 'red', 'female', '08CM64', 'Tesla', 'Model S')
(918773, 48, 65, 'black', 'red', 'female', '917UU3', 'Tesla', 'Model S')


**9. IDENTIFY OF WOMEN BASED ON SUSPECT INTERVIEW**

In [46]:

cursor=con.execute('''
SELECT * FROM person
where license_id=918773 or license_id=202298 or license_id=291182
''')
for row in cursor:
  print(row)

(78881, 'Red Korb', 918773, 107, 'Camerata Dr', 961388910)
(90700, 'Regina George', 291182, 332, 'Maple Ave', 337169072)
(99716, 'Miranda Priestly', 202298, 1883, 'Golden Ave', 987756388)


**10. CHECK THE PERSON_ID AND EVENT DETAILS AS PER SUSPECT INTERVIEW**

In [47]:
cursor=con.execute('''
SELECT person_id, count(*), event_name,date
FROM facebook_event_checkin
GROUP BY person_id
having count(*) = 3 AND event_name = "SQL Symphony Concert" AND date like "%201712%";
''')
for row in cursor:
  print(row)

(24556, 3, 'SQL Symphony Concert', 20171207)
(99716, 3, 'SQL Symphony Concert', 20171206)


**11. THE MASTERMIND BEHIND THIS MURDER**

In [48]:
cursor=con.execute('''
SELECT * from person
where id= 99716
''')
for row in cursor:
  print(row)

(99716, 'Miranda Priestly', 202298, 1883, 'Golden Ave', 987756388)
